In [1]:
import os
import sys
import tensorflow as tf

In [2]:
os.chdir("../")

In [3]:
##entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_file_path: Path

In [4]:
from src.utils.common import create_directories
from src.utils.common import read_yaml
from src.constants import constant_paths

CONFIG_FILE_PATH = constant_paths().get_paths()[0]
PARAMS_FILE_PATH = constant_paths().get_paths()[1]

[2023-07-28 12:42:27,656] 25 root INFO - Current working directory: c:\Users\pragn\Documents\MLOPS\final_CDC


In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        #self.params = read_yaml(params_file_path)

        #create_directories([self.config.artifacts_dir])

    def prepare_callbacks(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        create_directories([config.root_dir])

        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_file_path)

        create_directories([Path(model_checkpoint_dir),Path(config.tensorboard_root_log_dir)])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_file_path= Path(config.checkpoint_model_file_path)
        )
        #returning a class with loaded data path files
        return prepare_callbacks_config

Configuration is complete. Now creating required classes.

In [6]:
from src.logger import logging
from zipfile import ZipFile
import urllib.request as request
import time

In [12]:
class PrepareCallbacks:
    def __init__(self,config : PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)  

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_file_path),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [13]:
## Pipeline

from src.exception import CustomException

try:
    configmanager = ConfigurationManager()
    prepare_callbacks_config = configmanager.prepare_callbacks()
    prepare_callbacks = PrepareCallbacks(prepare_callbacks_config)
    callbacks_list = prepare_callbacks.get_tb_ckpt_callbacks()
    

except Exception as e:
    raise CustomException(e,sys)

[2023-07-28 12:47:46,498] 45 root INFO - yaml file: config\config.yaml loaded successfully
[2023-07-28 12:47:46,500] 64 root INFO - created directory at: artifacts/prepare_callbacks
[2023-07-28 12:47:46,501] 64 root INFO - created directory at: artifacts\prepare_callbacks\checkpoint_model
[2023-07-28 12:47:46,501] 64 root INFO - created directory at: artifacts\prepare_callbacks\tensorboard_root_log
